In [1]:
%load_ext autoreload
%autoreload 2

import sys  
sys.path.insert(1, '../')
sys.path.insert(1, '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src')
print(sys.path)

['/home/avijendra/nl2sql_lib/nl2sql-generic/notebooks', '/home/avijendra/nl2sql_lib/nl2sql-generic/nl2sql_src', '../', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/avijendra/fserv/lib/python3.11/site-packages']


In [2]:
# Uncomment the below commands if required to re-install the libraries
# %pip install faiss-cpu --quiet
# %pip install google.cloud --quiet
# %pip install cloud-sql-python-connector --quiet
# %pip install pg8000 --quiet
# %pip install langchain-google-vertexai

In [3]:
import numpy as np
import csv
import faiss
from pandas import DataFrame
from datetime import datetime
from tqdm import tqdm

In [4]:
# PROJECT_ID = ! gcloud config get project
from nl2sql_generic import Nl2sqlBq
PROJECT_ID = 'sl-test-project-363109'
LOCATION = 'us-central1'
DATASET_ID = 'zoominfo'

# Query and Embeddings in PostgreSQL

## Initialise

In [8]:
from nl2sql_src.nl2sql_query_embeddings import PgSqlEmb

# Make sure you update the following variables in-line with your project
PGPROJ = "sl-test-project-363109"
PGLOCATION = 'us-central1'
PGINSTANCE = "nl2sql-test"
PGDB = "test-db"
PGTABLE = 'documents' #'myqueries'
PGUSER = "postgres"
PGPWD = "nl2sql-test"
# INDX_FILE = 'saved_index_pgdata' # optoinal, considered as default

pge = PgSqlEmb(PGPROJ, PGLOCATION, PGINSTANCE, PGDB, PGUSER, PGPWD, pg_table=PGTABLE)
# mytable=PGTABLE

PGSQL - class Initiated


## Create Table

In [9]:
# Create Table in PostgresDB if not already existing
pge.create_table()

['sl-test-project-363109:us-central1:nl2sql-test']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "/home/avijendra/fserv/lib/python3.11/site-packages/aiohttp/connector.py", line 1173, in _create_direct_connection
    hosts = await asyncio.shield(host_resolved)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/fserv/lib/python3.11/site-packages/aiohttp/connector.py", line 884, in _resolve_host
    addrs = await self._resolver.resolve(host, port, family=self._family)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/avijendra/fserv/lib/python3.11/site-packages/aiohttp/resolver.py", line 33, in resolve
    infos = await self._loop.getaddrinfo(
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/asyncio/base_events.py", line 867, in getaddrinfo
    return await self.run_in_executor(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

## Insert records to PostgreSQL and update index

In [10]:
examples = []
with open('sample_sqls.csv', 'r') as f:
    samples_data = csv.reader(f)
    header = next(samples_data)
    for r in samples_data:
        examples.append(r)

for example in tqdm(examples):
    pge.insert_row(example[0], example[1])

100%|██████████| 19/19 [00:54<00:00,  2.88s/it]


In [12]:
new_question = "How many employees in California are from Livingston County?"
mapped_sql = "select count(*) from `sl-test-project-363109.zoominfo.zi-retail-companies-headquartered-california` where ZI_C_COMPANY_COUNTY = 'Livingston County'"
pge.insert_row(new_question, mapped_sql)

## Retrieve data 

In [24]:
# Retrieve the data from the PostgreDB and return a dataframe
def extract_data():
    tmp = pge.extract_data()
    df = DataFrame(tmp.fetchall())
    print(df)
    return df

df = extract_data()
df
headers = ["question", "sql"]
df.to_csv('sample_sqls.csv', columns=headers, index=False)

    q_id                                           question  \
0      1          How many people are enrolled in CalWorks?   
1      2            How many of them live in Nevada County?   
2      3  How has participation in CalWorks changed sinc...   
3      4   How do CalWorks program participation trends ...   
4      5  How have these race and ethnicity trends chang...   
5      6  Which county has the greatest proportion of Ca...   
6      7      What about three or more additional programs?   
7      8  Which programs have the highest co-enrollment ...   
8      9  Which programs have the highest co-enrollment ...   
9     10  What county has the greatest enrollment in Cal...   
10    11  Which five counties have the lowest number of ...   
11    12  How many Hispanic individuals are served acros...   
12    13                  What is the breakdown by program?   
13    14                        Has this changed over time?   
14    15                       Change over time by prog

## Search the VectorDB for closest matching queries

In [13]:
# Search for the closest matching queries
new_query = 'What is the total revenue for Hospitality industry'
output_json = pge.search_matching_queries(new_query)
output_json

Trying to read the index file ../../nl2sql-generic/nl2sql_src/cache_metadata/saved_index_pgdata


[{'question': ' How do CalWorks program participation trends differ by race and ethnicity?',
  'sql': "SELECT COALESCE(SUM(SAFE_CAST( White   AS INT64)), 0) AS calhhs_dashboard_2015_2020_annual_data_file_total_whites_1, COALESCE(SUM(SAFE_CAST( Black   AS INT64)), 0) AS calhhs_dashboard_2015_2020_annual_data_file_total_blacks_1, COALESCE(SUM(SAFE_CAST( Hispanic   AS INT64)), 0) AS calhhs_dashboard_2015_2020_annual_data_file_total_hispanics_1, COALESCE(SUM(SAFE_CAST( Asian_PI   AS INT64)), 0) AS calhhs_dashboard_2015_2020_annual_data_file_total_asian_1, COALESCE(SUM(SAFE_CAST( Native_American   AS INT64)), 0) AS calhhs_dashboard_2015_2020_annual_data_file_total_native_american_1 FROM `cdii-poc.HHS_Program_Counts.calhhs-dashboard-2015-2020-annual-data-file`  WHERE (Program ) = 'CalWorks';"},
 {'question': 'How many of them live in Nevada County?',
  'sql': "SELECT SUM(CAST(Person AS INT64)) AS total_beneficiaries FROM `cdii-poc.HHS_Program_Counts.calhhs-dashboard-2015-2020-annual-data-fil

## Clear VectorDB and re-create VectorDB from all data in PostgreSQL

In [11]:
pge.recreate_vectordb_index()

Number of elements :  14


## Clear all Data in DB

In [12]:
pge.empty_table()
df = extract_data()
df

Empty DataFrame
Columns: []
Index: []


""
